In [1]:
%matplotlib inline

import os
import sys
sys.path.append('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import glob
import random
import cv2
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F

import io
import evo
import evo.main_ape as main_ape
import evo.main_rpe as main_rpe

from tqdm import tqdm
from evo.core.metrics import PoseRelation, Unit
from evo.core.trajectory import PoseTrajectory3D
from evo.core import lie_algebra
from evo.tools.plot import PlotMode
from copy import deepcopy
from scipy.spatial.transform import Rotation
from PIL import Image
from matplotlib import pyplot as plt

from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images_ratio
from vggt.utils.pose_enc import pose_encoding_to_extri_intri
from vggt.utils.geometry import unproject_depth_map_to_point_map
from vggt.utils.helper import create_pixel_coordinate_grid, randomly_limit_trues
from vggt.dependency.track_predict import predict_tracks
from vggt.dependency.np_to_pycolmap import batch_np_matrix_to_pycolmap, batch_np_matrix_to_pycolmap_wo_track

from utils.umeyama import umeyama
from utils.cam_viz import create_interactive_camera_animation

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

In [8]:
import utils.colmap as colmap_utils
from utils.metric_torch import camera_to_rel_deg, calculate_auc_np

# Get image paths and preprocess them
data_dir = "../data/MipNeRF360"

# wald through folders under data_dir and calculate average performance
rot_error = []
translation_error = []
auc_30 = []

for subdir in os.listdir(data_dir):

    if not os.path.isdir(os.path.join(data_dir, subdir)):
        continue

    dust_dir = os.path.join(data_dir, subdir, "mast3r")

    if not os.path.exists(os.path.join(dust_dir, 'camera_poses.npy')) or \
         not os.path.exists(os.path.join(data_dir, subdir, 'pose_gt_train.npy')):
        print(f"Skipping {subdir} as required files are missing.")
        continue
    
    camtoworlds_train = np.load(os.path.join(dust_dir, 'camera_poses.npy'))
    camtoworlds_train_gt = np.load(os.path.join(data_dir, subdir, 'pose_gt_train.npy'))

    mast3r_se3 = torch.tensor(np.linalg.inv(camtoworlds_train), device=device)
    mast3r_gt_se3 = torch.tensor(np.linalg.inv(camtoworlds_train_gt), device=device)

    mast3r_se3[:, 3, :3] = mast3r_se3[:, :3, 3]
    mast3r_se3[:, :3, 3] = 0.0
    mast3r_gt_se3[:, 3, :3] = mast3r_gt_se3[:, :3, 3]
    mast3r_gt_se3[:, :3, 3] = 0.0

    # add alignment
    camera_centers_mast3r_gt = - (mast3r_gt_se3[:, :3, :3].cpu().numpy().transpose(0, 2, 1) @ mast3r_gt_se3[:, 3, :3][..., None].cpu().numpy()).squeeze(-1)
    camera_centers_mast3r_pred = - (mast3r_se3[:, :3, :3].cpu().numpy().transpose(0, 2, 1) @ mast3r_se3[:, 3, :3][..., None].cpu().numpy()).squeeze(-1)
    c, R, t = umeyama(camera_centers_mast3r_gt.T, camera_centers_mast3r_pred.T)
    camera_centers_mast3r_gt_aligned = (c * (R @ camera_centers_mast3r_gt.T) + t).T
    print("    --  Umeyama Scale: ", c)
    print("    --  Umeyama Rotation: \n", R)
    print("    --  Umeyama Translation: \n", t)

    ext_transform = np.eye(4)
    ext_transform[:3, :3] = R
    ext_transform[:3, 3:] = t
    ext_transform = np.linalg.inv(ext_transform)

    mast3r_gt_aligned = np.zeros((camtoworlds_train.shape[0], 4, 4))
    mast3r_gt_aligned[:, :3, :3] = mast3r_gt_se3[:, :3, :3].cpu().numpy()
    mast3r_gt_aligned[:, :3, 3] = mast3r_gt_se3[:, 3, :3].cpu().numpy() * c
    mast3r_gt_aligned[:, 3, 3] = 1.0
    mast3r_gt_aligned = np.einsum('bmn,bnk->bmk', mast3r_gt_aligned, ext_transform[None])

    mast3r_gt_se3_aligned = torch.eye(4, device=device).unsqueeze(0).repeat(camtoworlds_train.shape[0], 1, 1)
    mast3r_gt_se3_aligned[:, :3, :3] = torch.tensor(mast3r_gt_aligned[:, :3, :3], device=device)
    mast3r_gt_se3_aligned[:, 3, :3] = torch.tensor(mast3r_gt_aligned[:, :3, 3], device=device)

    rel_rangle_deg, rel_tangle_deg = camera_to_rel_deg(mast3r_se3, mast3r_gt_se3_aligned, device, 4)

    rError = rel_rangle_deg.cpu().numpy()
    tError = rel_tangle_deg.cpu().numpy()

    Auc_30 = calculate_auc_np(rError, tError, max_threshold=30)
    
    rot_error.append(rError.mean())
    translation_error.append(tError.mean())
    auc_30.append(Auc_30)

print("Average Rotation Error: ", np.mean(rot_error))
print("Average Translation Error: ", np.mean(translation_error))
print("Average AUC@30: ", np.mean(auc_30))

    --  Umeyama Scale:  0.49223298431022267
    --  Umeyama Rotation: 
 [[ 0.64462238 -0.27088347  0.71490148]
 [-0.00804973  0.93266472  0.36065458]
 [-0.76445875 -0.23824078  0.59903601]]
    --  Umeyama Translation: 
 [[-0.38132978]
 [-1.35942802]
 [ 0.30531174]]
Skipping flowers as required files are missing.
    --  Umeyama Scale:  0.4198601359739963
    --  Umeyama Rotation: 
 [[ 0.32209217 -0.05297554 -0.94522496]
 [-0.19672005  0.97289484 -0.12156008]
 [ 0.9260442   0.22509825  0.30294046]]
    --  Umeyama Translation: 
 [[ 1.3542229 ]
 [-1.88143672]
 [ 0.71802461]]
Skipping treehill as required files are missing.
    --  Umeyama Scale:  0.5685251628410306
    --  Umeyama Rotation: 
 [[-0.84703425  0.1114501  -0.51972286]
 [-0.45210337  0.36313373  0.81470021]
 [ 0.27952732  0.92504744 -0.25719974]]
    --  Umeyama Translation: 
 [[-1.1589561 ]
 [ 0.83238572]
 [ 0.73164876]]
    --  Umeyama Scale:  0.31234358905306103
    --  Umeyama Rotation: 
 [[-0.7627363  -0.39015712 -0.515

[[ 0.25057826  0.90359273  0.34746325]
 [-0.39124894  0.42281895 -0.81740345]
 [-0.88551387  0.06887891  0.45947893]]
    --  Umeyama Translation: 
 [[-0.6953709 ]
 [-2.4318474 ]
 [-3.26104026]]
    --  Umeyama Scale:  0.5508541530367199
    --  Umeyama Rotation: 
 [[-0.04744721 -0.25753836  0.96510246]
 [ 0.96990909 -0.24286435 -0.01712502]
 [ 0.23879933  0.93524911  0.26131203]]
    --  Umeyama Translation: 
 [[3.66612873]
 [0.01451746]
 [6.28388313]]
    --  Umeyama Scale:  0.5122194321156387
    --  Umeyama Rotation: 
 [[ 0.69930871  0.68414105 -0.20716747]
 [ 0.65734963 -0.50163019  0.56236876]
 [ 0.28081809 -0.52945083 -0.80051422]]
    --  Umeyama Translation: 
 [[1.91606598]
 [0.06981268]
 [1.88802055]]
